<a href="https://colab.research.google.com/github/meryemka112/Data-Mining-Labs/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

In [3]:
!pip install mmh3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.2 MB/s eta 0:00:00


In [4]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://cli.github.com/packages stable/main amd64 Packages [344 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,822 kB]
Get:14 https://r2u.st

In [5]:
# # Créer une session Spark
# spark = SparkSession.builder.appName("Exemple").getOrCreate()

# # Charger un fichier CSV
# df = spark.read.csv("metaverse_transactions_dataset.csv", header=True, inferSchema=True)

# # Afficher le schéma et une partie des données
# df.printSchema()
# df.show()



In [6]:
# type(df)

In [7]:
# df.select(['timestamp','amount']).show()

In [8]:
#spark.stop()

In [9]:
import pyspark
from pyspark.sql import DataFrame, SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col
import hashlib
from pyspark.sql.functions import lower, regexp_replace, split, udf
from pyspark.sql.types import ArrayType, StringType, LongType, IntegerType

Initialiser Apache Spark

In [10]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Document Similarity") \
    .getOrCreate()
sc = spark.sparkContext


In [11]:
import mmh3

**Classe Shingling**

In [28]:
class Shingling():
    def __init__(self, k=10):
        """
        k: length of shingles (in words)
        """
        self.k = k

    def clean_text(self, df: DataFrame, text_col: str) -> DataFrame:
        # Mettre en minuscule, supprimer caractères spéciaux, réduire espaces multiples
        df = df.withColumn(text_col, lower(col(text_col)))
        df = df.withColumn(text_col, regexp_replace(col(text_col), r"[^a-z0-9\s]", ""))
        df = df.withColumn(text_col, regexp_replace(col(text_col), r"\s+", " "))
        return df

    def tokenize(self, df: DataFrame, text_col: str) -> DataFrame:
        df = df.withColumn("tokens", split(col(text_col), "\\s+"))

        def clean_tokens(tokens):
            # Supprimer None et chaînes vides
            return [str(t).strip() for t in tokens if t and str(t).strip()]

        clean_udf = udf(clean_tokens, ArrayType(StringType())) #udf(...) transforme clean_tokens en fonction distribuée Spark.#Spark ne peut pas directement appliquer des fcts Python, besoin d'un wrapper -> udf
        df = df.withColumn("tokens", clean_udf(col("tokens")))
        return df

    def generate_shingles(self, df: DataFrame) -> DataFrame:
        k = self.k

        def shingle_function(tokens):
            tokens = [str(t).strip() for t in tokens if t and str(t).strip()]
            if len(tokens) < k:
                return []
            shingles = [" ".join(tokens[i:i+k]) for i in range(len(tokens) - k + 1)]
            # Supprimer shingles vides ou None
            return [s for s in shingles if s and s.strip()]

        shingle_udf = udf(shingle_function, ArrayType(StringType()))
        df = df.withColumn("shingles", shingle_udf(col("tokens")))
        return df

    def hash_shingles(self, df: DataFrame) -> DataFrame:
        def hash_list(shingles):
            hashed = []
            for s in shingles:
                if s and s.strip():
                    h = mmh3.hash64(s.strip(), signed=False)[0] #retourne entier non signé entre 0 et 2^64 -1
                    h = h % (2**63)  # forcer des entier entre 0 et 2^63 -1 #Sinon j'ai eu un overflow pour LongType et donc des valeurs NULL de hashage (mon prob initial)
                    #h = mmh3.hash("some string", signed=False)  # 32-bit unsigned integer
                    hashed.append(h)
            return hashed

        hash_udf = udf(hash_list, ArrayType(LongType()))   #longtype entre - 2^63 et 2^63 -1
        df = df.withColumn("hashed_shingles", hash_udf(col("shingles")))
        return df

    def transform(self, df: DataFrame, text_col: str) -> DataFrame:
        df = self.clean_text(df, text_col)
        df = self.tokenize(df, text_col)
        df = self.generate_shingles(df)
        df = self.hash_shingles(df)
        return df



In [29]:
data = [
    (1, "Spark is a distributed computing engine for large-scale data processing."),
    (2, "Hadoop is a framework for distributed computing and storage.")
]

df = spark.createDataFrame(data, ["id", "text"])

shingler = Shingling(k=3)
df_result = shingler.transform(df, "text")

df_result.select("id", "shingles", "hashed_shingles").show(truncate=False)


+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |shingles                                                                                                                                                                           |hashed_shingles                                                                                                                                                         |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------

In [ ]:
#Version python bof
# import hashlib

# class Shingling:
#     def __init__(self, k=10):
#         """
#         k: length of shingles (in words)
#         """
#         self.k = k

#     def tokenize(self, text):
#         """
#         Simple tokenization: split text into words
#         """
#         return text.lower().split()

#     def generate_shingles(self, tokens):
#         """
#         Generate k-word shingles from token list
#         """
#         shingles = []
#         for i in range(len(tokens) - self.k + 1):
#             shingle = ' '.join(tokens[i:i+self.k])
#             shingles.append(shingle)
#         return shingles

#     def hash_shingle(self, shingle):
#         """
#         Deterministic hash using MD5 truncated to 64 bits
#         """
#         h = int(hashlib.md5(shingle.encode('utf-8')).hexdigest()[:16], 16)
#         h = h % (2**63)  # ensure fits in 64-bit signed integer
#         return h

#     def shingle_document(self, text):
#         """
#         Complete pipeline: tokenize → generate shingles → hash shingles
#         Returns: ordered list of hashed shingles
#         """
#         tokens = self.tokenize(text)
#         shingles = self.generate_shingles(tokens)
#         hashed_shingles = [self.hash_shingle(s) for s in shingles]
#         return hashed_shingles


**Classe CompareSets**

In [30]:
class CompareSets():
  #méthode statique ici car n'accéde pas à l'attribut de l'objet, n'a besoin que de 2 arguments set1 et set 2
  @staticmethod
  def calculate_jaccard_similarity(set1,set2):
    if not set1 or not set2:
      return 0.0
    s1 = set(set1)
    s2 = set(set2)
    intersection = s1.intersection(s2)
    union = s1.union(s2)
    jaccard_similarity = len(intersection) / len(union)
    return jaccard_similarity


In [31]:
class MinHashing():
  def __init__(self,n):
    self.n = n
  def hash_set(self,shingle_set):
    n = self.n
    min_i = []
    for i in range(n):
      hashed = []
      for x in shingle_set:
        h = mmh3.hash(str(x), seed=i, signed=False)
        hashed.append(h)
      min_i.append(min(hashed))
    return min_i






In [32]:
shingles_doc1 = [6840193950337781150, 1695454882927409067, 7334696573069238721]
minhasher = MinHashing(n=5)
signature = minhasher.hash_set(shingles_doc1)

print(signature)
# Output example: [122343123, 2332231, 99221231, 28381223, 112392391]


[1325887520, 1238742759, 137587445, 39641790, 190228959]


In [33]:
class CompareSignatures():
  @staticmethod
  def compare(sig1,sig2):
    if len(sig1) != len(sig2):
      raise ValueError("Signatures must have the same length")

    matches = sum(1 for a,b in zip(sig1,sig2) if a == b)
    return matches /len(sig1)
